In [82]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import random
from time import sleep

In [2]:
h1 = {
'Cookie': 'bid=pWHRXljqlnw; _pk_id.100001.4cf6=1889b80fc0472f29.1700409010.; __utmc=30149280; __utmz=30149280.1700409010.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); __utmc=223695111; __utmz=223695111.1700409010.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); _ga=GA1.1.5521471.1700409010; ll="118282"; __utma=30149280.5521471.1700409010.1700409010.1700411523.2; __utma=223695111.1586942768.1700409010.1700409010.1700411523.2; _vwo_uuid_v2=D6D254F941FF811FF3C5BC45942B6A3E4|40c2965b4880f3c13ff94c0a7cdc0e98; _ga_RXNMP372GL=GS1.1.1700413503.2.0.1700413503.60.0.0; _pk_ref.100001.4cf6=%5B%22%22%2C%22%22%2C1700552142%2C%22https%3A%2F%2Fwww.google.com%2F%22%5D; _pk_ses.100001.4cf6=1; ap_v=0,6.0; dbcl2="276019366:j+LEQsNs20s"; ck=_dp9; frodotk_db="98c409fbb250fd78ca4c1df316f2f868"; push_noty_num=0; push_doumail_num=0',
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
'Accept-Encoding': 'gzip, deflate',
'Host': 'book.douban.com',
'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.4 Safari/605.1.15',
'Accept-Language': 'zh-CN,zh-Hans;q=0.9',
'Connection': 'keep-alive'
}

In [64]:
ip_list = []

with open('proxy.txt', 'r') as f:
    for line in f.readlines():
        ip_list.append(line.strip())

#获取ip代理
def getip():
    proxy= ip_list[random.randint(0,len(ip_list)-1)]
    proxy = proxy.replace("\n","")
    proxies={
        'http':'http://'+str(proxy),
        #'https':'https://'+str(proxy),
    }
    return proxies

def trans_star(v_str):
    if 'allstar10' in v_str:
        return '1'
    elif 'allstar20' in v_str:
        return '2'
    elif 'allstar30' in v_str:
        return '3'
    elif 'allstar40' in v_str:
        return '4'
    elif 'allstar50' in v_str:
        return '5'
    else:
        return 'NaN'

In [79]:
3*random()

1.6167300143069443

In [83]:


def get_short(book_url, max_page, data_path, file_name):
    df = pd.DataFrame()
    path =data_path + file_name + '.csv'
    if os.path.exists(path):
        print("file exists")
        return 0
    for i in range(0, max_page):
        sleep(3*random.random())
        url = book_url + 'comments?start=' + str(i*20) + '&limit=20&status=P&sort=new_score'
        res = requests.get(url, headers=h1, verify=False, proxies=getip())
        soup = BeautifulSoup(res.text, 'html.parser')
        user_names = soup.find_all('span', class_='comment-info')
        user_names = [i.find('a').text for i in user_names]
        stars = soup.find_all('span', class_='comment-info')
        
        stars = [i.find_all('span')[0]['class'] for i in stars]
        # print(stars)
        stars = [trans_star(i) for i in stars]
        comment_times = soup.find_all('a', class_='comment-time')
        
        comment_times = [i.text.strip() for i in comment_times]
        geo_ips = soup.find_all('span', class_='comment-location')
        # replace ''
        geo_ips = [i.text for i in geo_ips]
        geo_ips = ['unknown' if ip == '' else ip for ip in geo_ips]
        
        votes = soup.find_all('span', class_='vote-count')
        votes = [i.text for i in votes]
        shorts = soup.find_all('span', class_='short')
        shorts = [i.text for i in shorts]
        shorts = [i.replace(',', '，').replace(' ', '').replace('\n', '').replace('\r', '') for i in shorts]

        pd_data = {'user_name': user_names, 'star': stars, 'comment_time': comment_times, 'geo_ip': geo_ips, 'vote': votes, 'short': shorts}
        
        pd_data = pd.DataFrame(pd_data)
        # display(pd_data) # for testing
        df = pd.concat([df, pd_data], axis=0)
        print('Page ' + str(i+1) + ' is done!')

    if len(df) < 10:
        print('No data!')
        return 1

    df.to_csv(path, index=False)
    return 0

In [71]:
# test getshort()


# get_short("https://book.douban.com/subject/1007305/", 30, './data/', "红楼梦")
# get_short("https://book.douban.com/subject/4913064/", 30, './data/', "活着")
# get_short("https://book.douban.com/subject/6518605/", 30, './data/', "三体全集")


Page 1 is done!
Page 2 is done!
Page 3 is done!
Page 4 is done!
Page 5 is done!
Page 6 is done!
Page 7 is done!
Page 8 is done!
Page 9 is done!
Page 10 is done!
Page 11 is done!
Page 12 is done!
Page 13 is done!
Page 14 is done!
Page 15 is done!
Page 16 is done!
Page 17 is done!
Page 18 is done!
Page 19 is done!
Page 20 is done!
Page 21 is done!
Page 22 is done!
Page 23 is done!
Page 24 is done!
Page 25 is done!
Page 26 is done!
Page 27 is done!
Page 28 is done!
Page 29 is done!
Page 30 is done!


0

In [18]:
from IPython.display import display

# read book list
book_list = pd.read_csv('../analysis/BookDouban250.csv')

display(book_list.head(3))

# each line as a book, convert to list of dict
book_list = book_list.to_dict('records')

,Unnamed: 0,书名,豆瓣链接,作者,译者,出版社,出版日期,价格,评分,评分人数,一句话评价
0,0,红楼梦,https://book.douban.com/subject/1007305/,[清] 曹雪芹 著,NaN,人民文学出版社,1996-12,59.70元,9.6,413446,都云作者痴，谁解其中味？
1,1,活着,https://book.douban.com/subject/4913064/,余华,NaN,作家出版社,2012-8-1,20.00元,9.4,805477,生的苦难与伟大
2,2,1984,https://book.douban.com/subject/4820710/,[英] 乔治·奥威尔,刘绍铭,北京十月文艺出版社,2010-4-1,28.00,9.4,275446,栗树荫下，我出卖你，你出卖我


In [19]:
# disable warning
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [56]:
from tqdm.notebook import tqdm

In [70]:
book_list[3]

{'Unnamed: 0': 3,
 '书名': '三体全集',
 '豆瓣链接': 'https://book.douban.com/subject/6518605/',
 '作者': '刘慈欣 ',
 '译者': nan,
 '出版社': ' 重庆出版社 ',
 '出版日期': ' 2012-1 ',
 '价格': ' 168.00元',
 '评分': 9.5,
 '评分人数': 177655,
 '一句话评价': '地球往事三部曲'}

In [85]:

# get short for each book
for book in tqdm(book_list):
    book_url = book['豆瓣链接']# + "?start=0&limit=20&status=P&sort=new_score"
    max_page = 30
    data_path = './data/'
    file_name = book['书名'].replace(' ', '').split('/')[0].replace('\xa0', '')
    print(book_url, file_name)
    if book['书名'] == '1984':
        continue
    ret = get_short(book_url, max_page, data_path, file_name)
    
    if ret == 1:
        print("blocked!")
        break
    
    print(book['书名'] + ' is done!')


  0%|          | 0/250 [00:00<?, ?it/s]

https://book.douban.com/subject/1007305/ 红楼梦
file exists
红楼梦 is done!
https://book.douban.com/subject/4913064/ 活着
file exists
活着 is done!
https://book.douban.com/subject/4820710/ 1984
https://book.douban.com/subject/6518605/ 三体全集
file exists
三体全集 is done!
https://book.douban.com/subject/24531956/ 哈利·波特
file exists
哈利·波特 is done!
https://book.douban.com/subject/6082808/ 百年孤独
file exists
百年孤独 is done!
https://book.douban.com/subject/1068920/ 飘
file exists
飘 is done!
https://book.douban.com/subject/27614904/ 房思琪的初恋乐园
file exists
房思琪的初恋乐园 is done!
https://book.douban.com/subject/2035179/ 动物农场
file exists
动物农场 is done!
https://book.douban.com/subject/1019568/ 三国演义（全二册）
file exists
三国演义（全二册） is done!
https://book.douban.com/subject/1040211/ 福尔摩斯探案全集（上中下）
file exists
福尔摩斯探案全集（上中下） is done!
https://book.douban.com/subject/10554308/ 白夜行
file exists
白夜行 is done!
https://book.douban.com/subject/1084336/ 小王子
file exists
小王子 is done!
https://book.douban.com/subject/1060068/ 撒哈拉的故事
file exists
撒哈拉的故

TooManyRedirects: Exceeded 30 redirects.